# PyEddy: Getting Started Running the Code

In this tutorial you will learn: 

1. How to pick condensates to run in a cloud model 
2. What is $f_{sed}$ and what number is right for my model? 
3. How to compute initial Mie scattering parameters

If you are already familiar with these things, I recommend going moving to the next tutorial

In [1]:
from bokeh.io import output_notebook 
from bokeh.plotting import show, figure
from bokeh.palettes import Colorblind
output_notebook()
import numpy as np
import pandas as pd
import astropy.units as u
import justplotit

#here is pyeddy
import pyeddy as ed

Loading BokehJS ...

## Set up Calculation Parameters

In the very first step, we will set up our calculation run. This includes: 

1. Defining what gas condensates we want to include in the calculation. 

2. Defining the sedimentation efficiecy $f_{sed}$. 

3. Defining grid of particle radii for the Mie code

Let's briefly go through these one-by-one.

### Gas Condensates 

We highly recommend looking at your climate profile first to give an informed decision to the cloud model. First check out what current condensates are available

In [2]:
#you can print the available condensate species in the code
ed.available()

['CH4', 'Cr', 'Fe', 'H2O', 'KCl', 'Mg2SiO4', 'MgSiO3', 'MnS', 'NH3', 'ZnS']

Then you can let `pyeddy` give you a recommendation with some visualization aid

In [5]:
pressure = np.logspace(-5,3,30) #simple isotherml PT profile (kelvin)
temperature = np.zeros(30)+1600  
metallicity = 1 #atmospheric metallicity relative to Solar
mean_molecular_weight = 2.2 # atmospheric mean molecular weight

#get pyeddy recommendation for which gases to run
recommended = ed.recommend_gas(pressure, temperature, metallicity,mean_molecular_weight, 
                #Turn on plotting and add kwargs for bokeh.figure 
                 plot=True, y_axis_type='log',y_range=[1e2,1e-3],height=400, width=600,
                  y_axis_label='Pressure(bars)',x_axis_label='Temperature (K)')
#print the results
print(recommended)

['Cr', 'Fe', 'Mg2SiO4', 'MgSiO3']


### Compute Temperature Condensation Curves

`ed.recommend` is comparing your pressure-temperature curve against __condensation temperature curves__ for each species. If you are interested in computing your own you can do so!

In [4]:
cond_t_fig = figure(y_axis_type='log',y_range=[1e2,1e-3],height=400, width=400,
                  y_axis_label='Pressure(bars)',x_axis_label='Temperature (K)',
                   title='Condensation Temperatures')
metallicity = 1 
mean_molecular_weight = 2.2
for gas_name, c in zip(['Cr','MgSiO3','MnS'],Colorblind[8]): #case sensitive names
    #grab p,t from eddysed
    p,t = ed.condensation_t(gas_name, metallicity, mean_molecular_weight)
    #plot it up
    cond_t_fig.line(t,p, legend_label=gas_name,color=c,line_width=3)

show(cond_t_fig)    
    

### Defining $f_{sed}$

This parameter will set the vertical extent of our cloud. The general equation is: 

`condensate concentration (z)` $\propto$ `concentration at cloud base` $exp(-f_{sed}z/H)$

Therefore, high values of $f_{sed}$ deplete the cloud quickly, low values of $f_{sed}$ create large vertically thick clouds. __For context, it is thought that Hot Jupiters have fractional $f_{sed}$ values around 0.1, while Jupiter clouds can be parameterized with much larger values of 3-6.__

Here is a little schematic to show the general behavior of $f_{sed}$

In [8]:
#km, arbitrary planet scale height 
H = 27 
#define the cloud base 
cloud_base = 0.8 #bars  
#and some arbitrary height
z = np.linspace(0,300,100)#km 
pressure_grid=100*np.exp(-z/H)#bars 
#setup figure
toy_model = figure(x_axis_type='log',x_range=[1e-3,1.1],y_range=[1,1e-2],height=400, width=400,
                  y_axis_label='Pressure(bars)',x_axis_label='Arbitrary Fake Cloud Concentration Units',
                  title='F_sed Toy Model')
for fsed,c in zip([10,6,3,1,0.1],['red','orange','green','blue','purple']):
    cloud=np.zeros(100)+1e-13 #start with empty array 
    fake_cloud = np.exp(-fsed*z[pressure_grid<cloud_base]/H) #fill with exponential fake cloud 
    cloud[pressure_grid<cloud_base] = fake_cloud/np.max(fake_cloud) #fill cloud with fake cloud and normalize
    toy_model.line(cloud,pressure_grid,color=c,legend_label='F_sed='+str(fsed))
    toy_model.legend.location='bottom_left'
show(toy_model)

### Define Metallicity and Mean Molecular Weight

Although we are trying to make `pyeddy` as general as possible. 

## Set up Planet Parameters

In [ ]:
#set cloud species to condense
a = ed.Atmosphere(['Cr','MgSiO3'],fsed=0.01)

#set the planet gravity
a.get_gravity(gravity=357.00, gravity_unit=u.Unit('cm/(s**2)'))


In [ ]:
#set the pressure temperature profile and kzz
filename='/Users/nbatalh1/Documents/codes/TheOGTC/temp.dat'

a.get_pt(filename=filename  
    ,skiprows=3, delim_whitespace=True, header=None, names=["ind","pressure","temperature","kz"])

#get total opacity, single scattering, asymmetry, and individual optical depths 
opd, w0, g0, opd_gas= ed.justdoit(a, 
                                  directory='/Users/nbatalh1/Documents/data/eddy_optics')

## Using `PICASO` to visuzalize output

`PICASO` has a number of functions to visualize cloud output. We can use some eddysed functions to get the output into the specific format. 

In [ ]:
#get into DataFrame format
df = ed.picaso_format(opd,g0,w0)

In [ ]:
import picaso.justplotit as jpi

In [ ]:
show(jpi.plot_cld_input(nwno=196, nlayer=52,df=df,pressure=a.pressure))